In [56]:
import pandas as pd
import numpy as np
import os
import datetime
import sys
import bisect

sys.path.insert(0, '..')

from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

In [29]:
import umap
import umap.plot

In [30]:
raw_df = pd.read_hdf("../data/processed/summaries_zerg.hdf", "summaries")
raw_df = raw_df[raw_df.game_duration > 280]
raw_df.head()

,game_start,game_duration,self_won,self_name,self_race_is_protoss,self_race_is_zerg,self_race_is_terran,opponent_name,opponent_race_is_protoss,opponent_race_is_zerg,...,ZergMeleeWeaponsLevel2_start,ZergMeleeWeaponsLevel2_weight,ZergMissileWeaponsLevel1_start,ZergMissileWeaponsLevel1_weight,ZergMissileWeaponsLevel2_start,ZergMissileWeaponsLevel2_weight,Zergling_start,Zergling_weight,overlordspeed_start,overlordspeed_weight
045eeb4f0df4c471d53c4f1db5eea0294279e0d69fad7af2c2624e914806a2ba_1,2020-02-02 14:59:40,318,1.0,Ragnarok,0.0,1.0,0.0,DPGCure,0.0,0.0,...,1.0,0.0,1.000000,0.0,1.0,0.0,21.301775,2.234511,1.0,0.0
e5628efc602591574bb20c3bb83ee50cfc8705d3b6ea244aee60613b5a124fbc_0,2020-02-02 11:28:58,442,1.0,Ragnarok,0.0,1.0,0.0,TSGSolar,0.0,1.0,...,1.0,0.0,9.498681,1.0,1.0,0.0,19.780220,7.266902,1.0,0.0
e5628efc602591574bb20c3bb83ee50cfc8705d3b6ea244aee60613b5a124fbc_1,2020-02-02 11:28:58,442,0.0,TSGSolar,0.0,1.0,0.0,Ragnarok,0.0,1.0,...,1.0,0.0,1.000000,0.0,1.0,0.0,14.342629,9.691492,1.0,0.0
ec5459a85c6c9a22024fee37d2f1f23c4f15e0720dc14e6637fd076f528b86d4_0,2020-02-02 11:58:49,450,0.0,TSGSolar,0.0,1.0,0.0,Ragnarok,0.0,1.0,...,1.0,0.0,7.484407,1.0,1.0,0.0,21.176471,3.198894,1.0,0.0
ec5459a85c6c9a22024fee37d2f1f23c4f15e0720dc14e6637fd076f528b86d4_1,2020-02-02 11:58:49,450,1.0,Ragnarok,0.0,1.0,0.0,TSGSolar,0.0,1.0,...,1.0,0.0,7.708779,1.0,1.0,0.0,18.090452,9.517000,1.0,0.0


In [31]:
metadata_columns = """
self_won
game_start
game_duration
self_name self_race_is_protoss self_race_is_zerg self_race_is_terran
opponent_name
""".split()
weight_columns = [col for col in raw_df.columns if col.endswith("_weight")]

data_columns = [col for col in raw_df.columns if col not in metadata_columns and col not in weight_columns]

df = raw_df[data_columns]

encoding_dim = 8

In [32]:
#df.sort_values('DarkTemplar_start').tail(10)



In [33]:
input_data = Input(shape=(len(df.columns),))

encoded = Dense(64, activation='relu')(input_data)
encoded = Dense(24, activation='relu')(input_data)
#encoded = Dense(24, activation='relu')(input_data)

encoded = Dense(encoding_dim, activation='relu',
                    activity_regularizer=regularizers.l1(10e-3))(encoded)


#decoded = Dense(24, activation='relu')(encoded)
decoded = Dense(24, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(len(df.columns), activation='linear')(decoded)

autoencoder = Model(input_data, decoded)


In [34]:
encoder = Model(input_data, encoded)

In [35]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

In [36]:
x_all = df.sample(frac=1.0).values
num_samples = int(0.9 * x_all.shape[0])
x_train, x_test = x_all[:num_samples, :], x_all[num_samples:, :]
print("Training on {} samples. Testing on {}.".format(
    num_samples, x_all.shape[0] - num_samples))


Training on 2800 samples. Testing on 312.


In [37]:
autoencoder.fit(x_train, x_train,
                epochs=2500,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test))


Train on 2800 samples, validate on 312 samples
Epoch 1/2500
2800/2800 [==============================] - 0s 54us/step - loss: 12931889.4971 - val_loss: 12653872.0000
Epoch 2/2500
2800/2800 [==============================] - 0s 7us/step - loss: 12160153.5600 - val_loss: 10358260.0000
Epoch 3/2500
2800/2800 [==============================] - 0s 7us/step - loss: 7133958.4600 - val_loss: 3434517.0000
Epoch 4/2500
2800/2800 [==============================] - 0s 9us/step - loss: 2391666.8779 - val_loss: 1584914.0000
Epoch 5/2500
2800/2800 [==============================] - 0s 8us/step - loss: 1238225.7550 - val_loss: 1651390.7500
Epoch 6/2500
2800/2800 [==============================] - 0s 8us/step - loss: 1029065.0836 - val_loss: 435327.6875
Epoch 7/2500
2800/2800 [==============================] - 0s 7us/step - loss: 425262.0659 - val_loss: 507501.0625
Epoch 8/2500
2800/2800 [==============================] - 0s 8us/step - loss: 591944.1329 - val_loss: 266185.0312
Epoch 9/2500
2800/2800 [=

Epoch 73/2500
2800/2800 [==============================] - 0s 7us/step - loss: 55492.5602 - val_loss: 41825.1562
Epoch 74/2500
2800/2800 [==============================] - 0s 8us/step - loss: 50354.8836 - val_loss: 32622.9102
Epoch 75/2500
2800/2800 [==============================] - 0s 8us/step - loss: 45673.1603 - val_loss: 39621.2070
Epoch 76/2500
2800/2800 [==============================] - 0s 8us/step - loss: 50791.3674 - val_loss: 50049.3320
Epoch 77/2500
2800/2800 [==============================] - 0s 7us/step - loss: 55717.1566 - val_loss: 64605.6367
Epoch 78/2500
2800/2800 [==============================] - 0s 7us/step - loss: 74792.6049 - val_loss: 37678.0664
Epoch 79/2500
2800/2800 [==============================] - 0s 7us/step - loss: 49404.2507 - val_loss: 40510.7031
Epoch 80/2500
2800/2800 [==============================] - 0s 7us/step - loss: 54422.7636 - val_loss: 39392.0938
Epoch 81/2500
2800/2800 [==============================] - 0s 8us/step - loss: 53021.3464 - val_

2800/2800 [==============================] - 0s 6us/step - loss: 63874.6710 - val_loss: 41428.9297
Epoch 146/2500
2800/2800 [==============================] - 0s 7us/step - loss: 45516.9208 - val_loss: 43836.7266
Epoch 147/2500
2800/2800 [==============================] - 0s 6us/step - loss: 62238.2094 - val_loss: 34014.2031
Epoch 148/2500
2800/2800 [==============================] - 0s 7us/step - loss: 48880.3599 - val_loss: 42544.7812
Epoch 149/2500
2800/2800 [==============================] - 0s 6us/step - loss: 53228.1029 - val_loss: 44081.2773
Epoch 150/2500
2800/2800 [==============================] - 0s 6us/step - loss: 55177.2874 - val_loss: 51424.7891
Epoch 151/2500
2800/2800 [==============================] - 0s 7us/step - loss: 61270.2344 - val_loss: 55438.7891
Epoch 152/2500
2800/2800 [==============================] - 0s 6us/step - loss: 73592.5544 - val_loss: 42249.0039
Epoch 153/2500
2800/2800 [==============================] - 0s 8us/step - loss: 52786.8285 - val_loss: 

Epoch 217/2500
2800/2800 [==============================] - 0s 8us/step - loss: 59209.0167 - val_loss: 57063.0312
Epoch 218/2500
2800/2800 [==============================] - 0s 7us/step - loss: 62020.5179 - val_loss: 54888.3398
Epoch 219/2500
2800/2800 [==============================] - 0s 6us/step - loss: 68024.9596 - val_loss: 47062.8516
Epoch 220/2500
2800/2800 [==============================] - 0s 6us/step - loss: 58498.9370 - val_loss: 40306.0469
Epoch 221/2500
2800/2800 [==============================] - 0s 6us/step - loss: 49728.1466 - val_loss: 45188.4766
Epoch 222/2500
2800/2800 [==============================] - 0s 6us/step - loss: 48892.0461 - val_loss: 58122.9375
Epoch 223/2500
2800/2800 [==============================] - 0s 6us/step - loss: 69497.9800 - val_loss: 49971.6055
Epoch 224/2500
2800/2800 [==============================] - 0s 6us/step - loss: 60375.7763 - val_loss: 47160.3359
Epoch 225/2500
2800/2800 [==============================] - 0s 6us/step - loss: 58104.89

2800/2800 [==============================] - 0s 7us/step - loss: 62309.6748 - val_loss: 54236.0352
Epoch 289/2500
2800/2800 [==============================] - 0s 8us/step - loss: 65359.0002 - val_loss: 51981.2188
Epoch 290/2500
2800/2800 [==============================] - 0s 7us/step - loss: 59223.9571 - val_loss: 39280.7422
Epoch 291/2500
2800/2800 [==============================] - 0s 7us/step - loss: 45626.6517 - val_loss: 44088.7812
Epoch 292/2500
2800/2800 [==============================] - 0s 6us/step - loss: 50702.0492 - val_loss: 55774.2852
Epoch 293/2500
2800/2800 [==============================] - 0s 7us/step - loss: 59871.6042 - val_loss: 51547.8203
Epoch 294/2500
2800/2800 [==============================] - 0s 7us/step - loss: 60513.0559 - val_loss: 39713.7266
Epoch 295/2500
2800/2800 [==============================] - 0s 6us/step - loss: 48606.2229 - val_loss: 36488.9805
Epoch 296/2500
2800/2800 [==============================] - 0s 7us/step - loss: 44384.9698 - val_loss: 

...

2800/2800 [==============================] - 0s 8us/step - loss: 35921.2686 - val_loss: 37915.4180
Epoch 2214/2500
2800/2800 [==============================] - 0s 8us/step - loss: 46929.7347 - val_loss: 25806.3008
Epoch 2215/2500
2800/2800 [==============================] - ETA: 0s - loss: 27916.169 - 0s 8us/step - loss: 26587.2328 - val_loss: 38109.6602
Epoch 2216/2500
2800/2800 [==============================] - 0s 9us/step - loss: 48807.0870 - val_loss: 22215.4629
Epoch 2217/2500
2800/2800 [==============================] - 0s 6us/step - loss: 26973.1848 - val_loss: 27098.3828
Epoch 2218/2500
2800/2800 [==============================] - 0s 7us/step - loss: 27619.8263 - val_loss: 33633.4141
Epoch 2219/2500
2800/2800 [==============================] - 0s 8us/step - loss: 36810.7596 - val_loss: 27292.7559
Epoch 2220/2500
2800/2800 [==============================] - 0s 8us/step - loss: 32926.2953 - val_loss: 20232.9453
Epoch 2221/2500
2800/2800 [==============================] - 0s 8us/

2800/2800 [==============================] - 0s 6us/step - loss: 28311.5366 - val_loss: 29681.3984
Epoch 2285/2500
2800/2800 [==============================] - 0s 6us/step - loss: 36041.4967 - val_loss: 20149.8340
Epoch 2286/2500
2800/2800 [==============================] - 0s 7us/step - loss: 24915.5233 - val_loss: 21839.7324
Epoch 2287/2500
2800/2800 [==============================] - 0s 6us/step - loss: 29100.1657 - val_loss: 21520.8281
Epoch 2288/2500
2800/2800 [==============================] - 0s 7us/step - loss: 26960.3206 - val_loss: 23750.3809
Epoch 2289/2500
2800/2800 [==============================] - 0s 6us/step - loss: 29416.6871 - val_loss: 27243.2324
Epoch 2290/2500
2800/2800 [==============================] - 0s 7us/step - loss: 30297.2774 - val_loss: 30926.9023
Epoch 2291/2500
2800/2800 [==============================] - 0s 7us/step - loss: 32591.0161 - val_loss: 33721.2695
Epoch 2292/2500
2800/2800 [==============================] - 0s 7us/step - loss: 33076.2548 - va

2800/2800 [==============================] - 0s 7us/step - loss: 34755.3495 - val_loss: 38010.9492
Epoch 2356/2500
2800/2800 [==============================] - 0s 6us/step - loss: 36702.7748 - val_loss: 45152.7734
Epoch 2357/2500
2800/2800 [==============================] - 0s 6us/step - loss: 44193.7763 - val_loss: 36900.5820
Epoch 2358/2500
2800/2800 [==============================] - 0s 6us/step - loss: 39329.1087 - val_loss: 33754.0312
Epoch 2359/2500
2800/2800 [==============================] - 0s 6us/step - loss: 39672.3161 - val_loss: 30515.7539
Epoch 2360/2500
2800/2800 [==============================] - 0s 7us/step - loss: 35224.5824 - val_loss: 33405.1133
Epoch 2361/2500
2800/2800 [==============================] - 0s 7us/step - loss: 36603.5878 - val_loss: 28936.1914
Epoch 2362/2500
2800/2800 [==============================] - 0s 7us/step - loss: 34672.6111 - val_loss: 26955.1992
Epoch 2363/2500
2800/2800 [==============================] - 0s 8us/step - loss: 29556.0797 - va

2800/2800 [==============================] - 0s 6us/step - loss: 24314.0621 - val_loss: 23172.3301
Epoch 2427/2500
2800/2800 [==============================] - 0s 6us/step - loss: 26681.5750 - val_loss: 19439.5938
Epoch 2428/2500
2800/2800 [==============================] - 0s 7us/step - loss: 24510.4344 - val_loss: 24099.6914
Epoch 2429/2500
2800/2800 [==============================] - 0s 6us/step - loss: 28999.4100 - val_loss: 33424.7969
Epoch 2430/2500
2800/2800 [==============================] - 0s 6us/step - loss: 39153.4503 - val_loss: 29574.2168
Epoch 2431/2500
2800/2800 [==============================] - 0s 7us/step - loss: 32341.6083 - val_loss: 36576.3047
Epoch 2432/2500
2800/2800 [==============================] - 0s 7us/step - loss: 37817.4874 - val_loss: 29632.8477
Epoch 2433/2500
2800/2800 [==============================] - 0s 6us/step - loss: 36984.6132 - val_loss: 35390.8516
Epoch 2434/2500
2800/2800 [==============================] - 0s 7us/step - loss: 44799.7597 - va

2800/2800 [==============================] - 0s 6us/step - loss: 32135.2159 - val_loss: 21354.9688
Epoch 2498/2500
2800/2800 [==============================] - 0s 7us/step - loss: 26652.5672 - val_loss: 22266.9414
Epoch 2499/2500
2800/2800 [==============================] - 0s 6us/step - loss: 28188.3235 - val_loss: 26247.0156
Epoch 2500/2500
2800/2800 [==============================] - 0s 6us/step - loss: 31550.9950 - val_loss: 29037.9414


In [54]:
cluster_games_df = raw_df[raw_df.game_start > datetime.datetime(2020, 1, 1)]
encoded_space = encoder.predict(cluster_games_df[data_columns].values)

In [55]:
num_elements, _ = encoded_space.shape
distance_matrix = np.ndarray(shape=(num_elements, num_elements), dtype=np.float32)
single_row_item = np.ndarray(shape=encoded_space.shape, dtype=encoded_space.dtype)

for i in range(num_elements):
    single_row_item[:] = encoded_space[i, :] # broadcast single row to whole matrix

    diffs = encoded_space - single_row_item
    distance_matrix[i, :] = np.einsum("ij,ij->i", diffs, diffs)

In [51]:
distance_series = pd.Series(distance_matrix.reshape(-1))
percentiles = distance_series.quantile([
    0.05, 0.10, 0.15, 0.20, 0.25
])

0.05    0.030131
0.10    0.126705
0.15    0.304774
0.20    0.595565
0.25    1.062973
dtype: float64

In [62]:
def creativity_score(row):
    base = 0.75
    too_far = len(percentiles)
    return sum(
        base ** exponent if exponent < too_far else 0
        for exponent in (
            np.searchsorted(percentiles, x) for x in row
        )
    )

In [ ]:
cluster_games_df['creativity'] = np.apply_along_axis(creativity_score, 1, distance_matrix)